# Задание
Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события

- event - событие

app_install – установка приложения
app_start – открыть приложения
registration – зарегистрироваться 
search – перейти на страницу поиска товаров (каталог)
open_item – открыть товар
choose_item – отправить товар в корзину
tap_basket – перейти в корзину
purchase – подтверждение покупки
- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

yandex-direct – Яндекс директ
google_ads – реклама в Google
vk_ads – реклама в ВК
instagram_ads – реклама в instagram
facebook_ads – реклама в facebook
referral – акция «приведи друга»

In [1]:
#Импортируем библиотеки
import pandas as pd
import seaborn as sns

In [2]:
#Считываем файл, переведем столбец "date" в формат даты
case = pd.read_csv("D://KARPOV/KC_case_data.csv", encoding='utf-8', parse_dates=['date'])


In [3]:
case.head(3)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal


# 1.
Найдите MAU февраля

In [4]:
case.groupby(pd.Grouper(key='date', freq='1M')).agg({'device_id':'nunique'})

,device_id
date,
2020-01-31,99161
2020-02-29,75032
2020-03-31,74623


#### MAU февраля - 75032

# 2. 
Найдите количество установок в январе

In [5]:
case.event.value_counts()

app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: event, dtype: int64

In [6]:
#Сделаем сводную таблицу 
case.groupby(['event',pd.Grouper(key='date', freq='1M')]).agg(count=('event','count')).reset_index().pivot(index='date', columns='event', values='count')

event,app_install,app_start,choose_item,purchase,register,search,tap_basket
date,,,,,,,
2020-01-31,80297,268471,196137,59208,42946,254732,140616
2020-02-29,38078,243694,174911,44594,20578,230689,122181
2020-03-31,36222,236540,167621,37581,14786,223218,114868


#### Видим, что количество установок в январе = 80297

# 3. 
Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [7]:
case_new=case.query('event == "app_install" or event == "purchase"').groupby(['device_id', 'date','event']).agg(count=('event','count')).sort_values('date')

In [8]:
case_new.head(3)

,,,count
device_id,date,event,
936996,2020-01-01,purchase,1
32817948,2020-01-01,app_install,1
25746914,2020-01-01,app_install,1


In [9]:
case_new=case_new.reset_index().groupby(['device_id', 'event']).agg(date_action=('date','first')).sort_values('device_id').reset_index()

In [10]:
case_new=case_new.pivot(index='device_id', columns='event', values='date_action')
case_new.head(5)

event,app_install,purchase
device_id,,
4013,2020-01-15,NaT
4014,NaT,2020-02-01
4016,2020-01-04,NaT
4046,2020-01-04,2020-01-04
4060,NaT,2020-03-07


Получаем таблицу, где видим, что, например, device_id = 4046 зарегистрировался и в тот же день совершил первую покупку.
При этом есть пользователи, которые совершили покупку, но не установили приложение. 
Это объясняется тем, что у них уже оно было уставновлено.

In [11]:
case_new['days_diff']=case_new['purchase']-case_new['app_install']

In [12]:
case_new['seven_days'] =case_new['days_diff'] <= '7 days'
case_new.head(5)

event,app_install,purchase,days_diff,seven_days
device_id,,,,
4013,2020-01-15,NaT,NaT,False
4014,NaT,2020-02-01,NaT,False
4016,2020-01-04,NaT,NaT,False
4046,2020-01-04,2020-01-04,0 days,True
4060,NaT,2020-03-07,NaT,False


In [13]:
case_new = case_new.groupby('app_install').seven_days.agg({'count', 'sum'})
case_new.head(5)

,sum,count
app_install,,
2020-01-01,1408,3579
2020-01-02,1186,3144
2020-01-03,834,2402
2020-01-04,639,1831
2020-01-05,587,1671


In [14]:
#Посчитаем конверсию
case_new['cr']= (case_new['sum'] / case_new['count']).mul(100).round(1)
case_new.sort_values('cr', ascending=False)

,sum,count,cr
app_install,,,
2020-01-01,1408,3579,39.3
2020-01-09,558,1424,39.2
2020-01-15,1650,4310,38.3
2020-01-14,1973,5173,38.1
2020-01-02,1186,3144,37.7
...,...,...,...
2020-03-18,235,1171,20.1
2020-03-29,223,1117,20.0
2020-03-22,251,1261,19.9


#### Наибольшая конверсия была для когорты 2020-01-01 и составила 39,1%

# 4.
С какого платного маркетингового канала пришло больше всего новых пользователей? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

NB! В этом и подобном ему заданиях писать надо только название, без кавычек.



In [15]:
case.query('event == "app_install"')\
.groupby('utm_source')\
.agg({'device_id':'nunique'})\
.sort_values('device_id', ascending=False).iloc[1:]

,device_id
utm_source,
yandex-direct,29368
google_ads,26286
vk_ads,23189
instagram_ads,20096
facebook_ads,13916
referal,9282


#### Больше всего новых пользователей пришло к нам из Яндекс.Директ (29368 уникальных id)

# 5.
Количество установок в январе?

In [16]:
#Отберем установки и сгруппируем по месяцам
case.query('event == "app_install"').groupby(pd.Grouper(key='date', freq="M")).agg({'device_id':'nunique'})

,device_id
date,
2020-01-31,80297
2020-02-29,38078
2020-03-31,36222


#### Количество установок в январе составило 80297

# 6.
Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: 
- «Поиск», 
- «Добавление товара»,  
- «Переход в корзину», 
- «Регистрация».

In [17]:
funnel=case.iloc[:, [0,1, 4, 7]].query('event =="register"').sort_values('date').drop_duplicates('device_id')
funnel.head(3)

,date,event,device_id,utm_source
2669658,2020-01-01,register,294193,instagram_ads
2670356,2020-01-01,register,21780,yandex-direct
2670357,2020-01-01,register,2470180,google_ads


In [18]:
case['first_reg']=case['device_id'].map(funnel.set_index('device_id')['date'])
case.head(5)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,first_reg
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT


In [19]:
#Если пользователь был уже зарегистрирован, то дата первой регистрации будет меньше даты совершения события 
already_registered = case[case['first_reg'] < case['date']]

In [20]:
already_registered=already_registered.groupby('event')['device_id'].nunique()

In [21]:
already_registered

event
app_start      40991
choose_item    37926
purchase       24880
search         40482
tap_basket     34517
Name: device_id, dtype: int64

In [22]:
print('Конверсия в поиск из открытия -', round((already_registered[3]/already_registered[0]) * 100, 2))
print('Конверсия в добавление товара из поиска -', round((already_registered[1]/already_registered[3]) * 100, 2))
print('Конверсия в переход в корзину из добавления товара -', round((already_registered[4]/already_registered[1]) * 100, 2))
print('Конверсия в покупку из перехода в корзину -', round((already_registered[2]/already_registered[4]) * 100, 2))

Конверсия в поиск из открытия - 98.76
Конверсия в добавление товара из поиска - 93.69
Конверсия в переход в корзину из добавления товара - 91.01
Конверсия в покупку из перехода в корзину - 72.08


#### Как мы видим, самая низкая коверсия наблюдается из этапа "Переход к корзину" в  этап "Покупка".

# 7.
Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

Будем считать: 
CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [23]:
#Отберем нужные столбцы, далее сделаем сводную таблицу, где values = кол-во уникальных device_id
source=case.iloc[:, [0, 1, 4, 7]]
source=source.query('event == ["app_start","purchase"]').query('utm_source != "-"').pivot_table( 
                    values  = 'device_id', 
                    index   = 'utm_source',
                    columns = 'event', 
                    aggfunc = pd.Series.nunique
                    )
source

event,app_start,purchase
utm_source,,
facebook_ads,25957,9017
google_ads,38092,11339
instagram_ads,31045,10762
referal,15924,6362
vk_ads,34079,12364
yandex-direct,40707,12028


In [24]:
#Посчитаем конверсию
(source.purchase / (source.app_start )).to_frame().sort_values(0, ascending=False).reset_index().rename(columns={0: "CR"})

,utm_source,CR
0,referal,0.399523
1,vk_ads,0.362804
2,facebook_ads,0.347382
3,instagram_ads,0.346658
4,google_ads,0.297674
5,yandex-direct,0.295477


In [25]:
#Найдем канал с минимальной конверсией
(source.purchase / (source.app_start )).idxmin()

'yandex-direct'

#### Получается, что самая низкая конверсия среди тех, кто пришел из "Яндекс.Директ"

# 8.
Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [26]:
#Отберем строки с покупками, а затем нужные столбцы из датасета
purchase = case[case['event'] == 'purchase']
purchase=purchase.iloc[:, [0, 2, 4, 7]]
purchase

,date,purchase_sum,device_id,utm_source
7691,2020-01-01,265.5,6081155,-
7692,2020-01-01,374.0,77448,yandex-direct
7693,2020-01-01,98.5,7458607,instagram_ads
7694,2020-01-01,304.5,3520947,instagram_ads
7695,2020-01-01,153.5,1170605,google_ads
...,...,...,...,...
2507469,2020-03-31,6546.5,11864116,google_ads
2507470,2020-03-31,3495.0,3403969,google_ads
2507471,2020-03-31,2857.5,26120362,google_ads
2507472,2020-03-31,5671.5,22011351,yandex-direct


In [27]:
#Отсортируем по дате asc, т.к. в задании требуется первый чек, а затем удалим дубликаты id
purchase=purchase.sort_values('date').drop_duplicates('device_id')

In [28]:
#Теперь сгруппируем по каналам и вычислим медиану по каждому
purchase.groupby('utm_source').median('purchase_sum').iloc[:, [0]]

,purchase_sum
utm_source,
-,398.5
facebook_ads,389.0
google_ads,390.5
instagram_ads,393.5
referal,395.5
vk_ads,393.0
yandex-direct,392.5


#### Выходит, что медианная сумма покупки была больше всех у пользователей, пришедших с реферальной программы (395.5). А самый высокий медианный чек наблюдается с прямых продаж (-) (398.5р)

# 9.
Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения имеет самый высокий ROMI? 

In [34]:
romi=case.groupby('utm_source').agg({'purchase_sum':'sum', 'device_id':'count'})

In [30]:
romi=romi.reset_index()
romi

,utm_source,purchase_sum,device_id
0,-,21449749.5,610458
1,facebook_ads,12249901.0,336953
2,google_ads,12868276.0,374257
3,instagram_ads,14546969.0,382891
4,referal,8837044.5,234721
5,vk_ads,16389652.5,418456
6,yandex-direct,13915368.0,390232


In [31]:
#Добавим столбец expenditure с данными расходов из условия
romi['expenditure']=pd.DataFrame(data={'expenditure': ['NaN', 8590498, 10534878, 8561626,2*636000,9553531,10491707]})
romi=romi[1:]

In [32]:
#Добавим столбец romi, вычитаем значения показателя для каждого канала
romi['romi'] = (romi['purchase_sum']/romi['expenditure'] - 1)
romi

<ipython-input-32-38f65881fba3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  romi['romi'] = (romi['purchase_sum']/romi['expenditure'] - 1)


,utm_source,purchase_sum,device_id,expenditure,romi
1,facebook_ads,12249901.0,336953,8590498,0.425983
2,google_ads,12868276.0,374257,10534878,0.221493
3,instagram_ads,14546969.0,382891,8561626,0.69909
4,referal,8837044.5,234721,1272000,5.94736
5,vk_ads,16389652.5,418456,9553531,0.71556
6,yandex-direct,13915368.0,390232,10491707,0.326321


In [33]:
romi.sort_values('romi', ascending=False)

,utm_source,purchase_sum,device_id,expenditure,romi
4,referal,8837044.5,234721,1272000,5.94736
5,vk_ads,16389652.5,418456,9553531,0.71556
3,instagram_ads,14546969.0,382891,8561626,0.69909
1,facebook_ads,12249901.0,336953,8590498,0.425983
6,yandex-direct,13915368.0,390232,10491707,0.326321
2,google_ads,12868276.0,374257,10534878,0.221493


#### Среди платных каналов - VK имеет наибольший показатель ROMI